In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import keras
from keras import layers
import numpy as np

In [0]:
from tqdm import tqdm

In [0]:
latent_dim = 32
height = 32
width = 32
channels = 3

#Generator

In [0]:
generator_input = keras.Input(shape=(latent_dim, ))

x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)

generator = keras.models.Model(generator_input, x)

In [0]:
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_34 (LeakyReLU)   (None, 32768)             0         
_________________________________________________________________
reshape_7 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_35 (LeakyReLU)   (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 32, 32, 256)       1048832   
__________

#Discriminator

In [0]:
discriminator_input = layers.Input(shape=(height, width, channels))

x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x)

x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)

In [0]:
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_39 (LeakyReLU)   (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_40 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_41 (LeakyReLU)   (None, 6, 6, 128)         0         
__________

In [0]:
discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0008, clipvalue=1.0, decay=1e-8)

In [0]:
discriminator.compile(
    optimizer = discriminator_optimizer,
    loss = 'binary_crossentropy'
)

#Adversarial Net

In [0]:
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim, ))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)

In [0]:
gan.compile(
    optimizer = gan_optimizer,
    loss = 'binary_crossentropy'
)

#Train

In [0]:
import os
from keras.preprocessing import image

In [0]:
(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()
x_train = x_train[y_train.flatten() == 6]
x_train = x_train.reshape((x_train.shape[0], ) + (height, width, channels)).astype('float32') / 255.

In [0]:
iterations = 100000
batch_size = 20

In [0]:
save_dir = '/content/drive/My Drive/ML/keras_book/Generate/GAN'

In [0]:
start = 0
for step in tqdm(range(iterations)):
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    generated_images = generator.predict(random_latent_vectors)
    
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])
    
    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(labels.shape)
    
    d_loss = discriminator.train_on_batch(combined_images, labels)
    
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    
    misleading_targets = np.zeros((batch_size, 1))
    
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0
        
    if step % 1000 == 0:
        gan.save_weights('gan.h5')
        
        print('discriminator loss at step %s: %s' %(step, d_loss))
        print('adversarial loss at step %s: %s' %(step, a_loss))
        
        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'generated' + str(step) + '.png'))
        
        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'real' + str(step) + '.png'))







  0%|          | 0/100000 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'






  0%|          | 1/100000 [00:00<5:53:43,  4.71it/s]





  0%|          | 2/100000 [00:00<4:58:05,  5.59it/s]

discriminator loss at step 0: 0.70923126
adversarial loss at step 0: 0.7505838








  0%|          | 3/100000 [00:00<4:20:47,  6.39it/s]





  0%|          | 4/100000 [00:00<3:55:26,  7.08it/s]





  0%|          | 5/100000 [00:00<3:37:00,  7.68it/s]





  0%|          | 6/100000 [00:00<3:24:34,  8.15it/s]





  0%|          | 7/100000 [00:00<3:14:20,  8.58it/s]





  0%|          | 8/100000 [00:00<3:07:50,  8.87it/s]





  0%|          | 9/100000 [00:01<3:02:41,  9.12it/s]





  0%|          | 10/100000 [00:01<2:59:40,  9.28it/s]





  0%|          | 11/100000 [00:01<2:58:13,  9.35it/s]





  0%|          | 12/100000 [00:01<2:56:58,  9.42it/s]





  0%|          | 13/100000 [00:01<2:55:46,  9.48it/s]





  0%|          | 14/100000 [00:01<2:56:36,  9.44it/s]





  0%|          | 15/100000 [00:01<2:56:52,  9.42it/s]





  0%|          | 16/100000 [00:01<2:56:12,  9.46it/s]





  0%|          | 17/100000 [00:01<2:55:25,  9.50it/s]





  0%|          | 18/100000 [00:01<2:55:28,  9.50it/s]





  0%|          | 19/100000 [00:02<2:53:58,  9.58it/s]



